### 0. imports

In [6]:
# data science
import scipy as sp
# type hints
from scipy.sparse import coo_matrix


### 1.1. generate sparse matrices

In [7]:
M = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'coo',
    dtype = float,
    random_state = 42
)

In [8]:
M

<20000x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 4000000 stored elements in COOrdinate format>

### 2.1. setting rows to zero

📚 references:

1. https://stackoverflow.com/q/19784868
2. https://stackoverflow.com/q/12129948
3. https://stackoverflow.com/q/28427236
4. https://stackoverflow.com/q/15896588



[`%%timeit` magic documentation](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

In [5]:
%%timeit


UsageError: %%timeit is a cell magic, but the cell body is empty. Did you mean the line magic %timeit (single %)?
